# Exercises

Data Acquisition

These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

This exercises use the cases, dept, and source tables from the 311_data on the Codeup MySQL server.

- 1)Read the case, department, and source data into their own spark dataframes.
- 2)Let's see how writing to the local disk works in spark:
>Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

>Inspect your folder structure. What do you notice?

- 3)Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.
>PART 2
- 1)How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?
- 2)How many Stray Animal cases are there?
- 3)How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?
- 4)Convert the council_district column to a string column.
- 5)Extract the year from the case_closed_date column.
- 6)Convert num_days_late from days to hours in new columns num_hours_late.
- 7)Join the case data with the source and department data.
- 8)Are there any cases that do not have a request source?
- 9)What are the top 10 service request types in terms of number of requests?
- 10)What are the top 10 service request types in terms of average days late?
- 11)Does number of days late depend on department?
- 12)How do number of days late depend on department and request type?


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

# 1)Read the case, department, and source data into their own spark dataframes.

In [6]:
case = spark.read.csv("cases.csv", sep=",", header=True, inferSchema=True)
case.show()

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO|       -998.5087616|        YES|Field Operations|        Stray Animal|               999|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|       -2.012604167|        YES|     Storm Water|Remova

In [7]:
dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)
dept.show()

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|                NULL|  DSD/Code Enforcement|                YES|
|   Dangerous Premise|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Dangerous Premise...|Code Enforcement ...|

In [8]:
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)
source.show()

+-----+---------+--------------------+
|index|source_id|     source_username|
+-----+---------+--------------------+
|    0|   100137|    Merlene Blodgett|
|    1|   103582|         Carmen Cura|
|    2|   106463|     Richard Sanchez|
|    3|   119403|      Betty De Hoyos|
|    4|   119555|      Socorro Quiara|
|    5|   119868| Michelle San Miguel|
|    6|   120752|      Eva T. Kleiber|
|    7|   124405|           Lori Lara|
|    8|   132408|       Leonard Silva|
|    9|   135723|        Amy Cardenas|
|   10|   136202|    Michelle Urrutia|
|   11|   136979|      Leticia Garcia|
|   12|   137943|    Pamela K. Baccus|
|   13|   138605|        Marisa Ozuna|
|   14|   138650|      Kimberly Green|
|   15|   138650|Kimberly Green-Woods|
|   16|   138793| Guadalupe Rodriguez|
|   17|   138810|       Tawona Martin|
|   18|   139342|     Jessica Mendoza|
|   19|   139344|        Isis Mendoza|
+-----+---------+--------------------+
only showing top 20 rows



# 3)Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [18]:
case = case.withColumnRenamed('SLA_due_date', 'case_due_date')

In [19]:
case = case.withColumn('case_closed', expr('case_closed == "YES"'))\
           .withColumn('case_late', expr('case_late == "YES"'))

+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|     true|
+-----------+---------+
only showing top 5 rows



In [20]:
fmt = 'M/d/yy H:mm'

case = case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))\
           .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))\
           .withColumn("case_due_date", to_timestamp("case_due_date", fmt))

case.select('case_opened_date', 'case_closed_date', 'case_due_date').show(5)

+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|
|2018-01-01 00:46:00|2018-01-03 08:11:00|2018-01-05 08:30:00|
|2018-01-01 00:48:00|2018-01-02 07:57:00|2018-01-05 08:30:00|
|2018-01-01 01:29:00|2018-01-02 08:13:00|2018-01-17 08:30:00|
|2018-01-01 01:34:00|2018-01-01 13:29:00|2018-01-01 04:34:00|
+-------------------+-------------------+-------------------+
only showing top 5 rows



# 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [21]:
case.filter('! case_closed')\
    .select('case_opened_date', 'case_due_date', 'num_days_late')\
    .sort(desc('num_days_late'))\
    .show(5)

+-------------------+-------------+-------------+
|   case_opened_date|case_due_date|num_days_late|
+-------------------+-------------+-------------+
|2017-09-11 16:24:00|         null|         NULL|
|2017-08-21 15:22:00|         null|         NULL|
|2017-08-23 15:47:00|         null|         NULL|
|2017-09-06 15:17:00|         null|         NULL|
|2017-08-01 10:43:00|         null|         NULL|
+-------------------+-------------+-------------+
only showing top 5 rows



# 2)How many Stray Animal cases are there?

In [10]:
case.filter(case.service_request_type == lit('Stray Animal')).count()

26760

#  3)How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?



In [ ]:
case.filter(case.dept_division == 'Field Operations')\
    .filter(case.service_request_type != 'Officer Standby')\
    .count()

#  4)Convert the council_district column to a string column.



In [ ]:
case = case.withColumn('council_district', col('council_district').cast('string'))

#  5)Extract the year from the case_closed_date column.



In [ ]:
case.withColumn('year', year('case_closed_date'))\
    .select('service_request_type', 'case_closed_date', 'year')\
    .show(5)

#  6)Convert num_days_late from days to hours in new columns num_hours_late.



In [ ]:
case.withColumn('num_hours_late', round(expr('num_days_late * 24'), 1))\
    .select('num_days_late', 'num_hours_late')\
    .show(10)

#  7)Join the case data with the source and department data.



In [ ]:
case = case.join(source, 'source_id', 'left')\
           .drop(source.source_id)

case.show(2, vertical=True)

# 8)Are there any cases that do not have a request source?



In [11]:
case.filter(case.source_id == 'null').count()

0

# 9)What are the top 10 service request types in terms of number of requests?

In [12]:
case.groupBy('service_request_type')\
    .count()\
    .sort(desc('count'))\
    .show(10)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|86855|
|Overgrown Yard/Trash|65895|
|        Bandit Signs|32910|
|        Damaged Cart|30338|
|Front Or Side Yar...|28794|
|        Stray Animal|26760|
|Aggressive Animal...|24882|
|Cart Exchange Req...|22024|
|Junk Vehicle On P...|21473|
|     Pot Hole Repair|20616|
+--------------------+-----+
only showing top 10 rows



# 10)What are the top 10 service request types in terms of average days late?

In [13]:
case.groupBy('service_request_type')\
    .agg(round(mean('num_days_late'), 1).alias('avg_late'))\
    .sort(desc('avg_late'))\
    .show(10)

+--------------------+--------+
|service_request_type|avg_late|
+--------------------+--------+
|  Zoning: Junk Yards|   176.0|
|Labeling for Used...|   162.4|
|Record Keeping of...|   154.0|
|Signage Requied f...|   151.6|
|Storage of Used M...|   142.1|
|Zoning: Recycle Yard|   135.9|
|Donation Containe...|   131.8|
|License Requied U...|   128.8|
|Traffic Signal Gr...|    77.9|
|           Complaint|    72.5|
+--------------------+--------+
only showing top 10 rows



# 11)How do number of days late depend on department and request type?

In [ ]:
case.groupBy('department', 'service_request_type')\
    .agg(round(avg('num_days_late'), 1).alias('avg_days_late'))\
    .sort('department')\
    .show(50)